In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
MODEL="gpt-3.5-turbo"
MODEL="llama2"
# MODEL="mixtral:8x7b"

In [2]:
from langchain_openai.chat_models import ChatOpenAI 
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model=ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
    embeddings=OpenAIEmbeddings()
else:
    model=Ollama(model=MODEL)
    embeddings=OllamaEmbeddings()
model.invoke("Tell me a joke")

c:\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
chain=model | parser
chain.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("sanction-letter-A2312180286.pdf")
pages=loader.load_and_split()
pages

[Document(page_content='Dear Sir/Madam,A2312180286\nTo,\nAtharva Sadashiv Tapkir & Sadashiv Pandharinath Tapkir & Smita Sadashiv Tapkir\nAddress : Sr No 52 1 Building No B 7 Flat No 05 Shantiban Society Near Eklavya Collage Tc Kothrud Po Serviceman Colony,\nPune, 411038, Maharashtra\nSubject: Education Loan Sanction Letter\nBased upon and relying on the facts and information provided by you in your Application Form No. A2312180286 dated 18/12/2023 seeking grant of an\nEducation Loan ("loan"), we have pleasure in conveying you our sanction of an amount of Rs. 50,00,000/- (Rupees Fifty Lakh only) (equivalent to USD\n60,233.71 approx. at exchange rate of 1 USD : 83.01 INR. ) on following terms:\nThe Educational Loan availability is subject to satisfactory completion of loan documentation. This letter of offer shall stand unilaterally revoked and\ncancelled and shall be absolutely null and void even after acceptance by you, if:\nHDFC Credila Financial Services Limited\nAuthorized signatory

In [5]:
from langchain.prompts import PromptTemplate

template=""" 
Answer the question based on the context below. 
If you can't answer the question, reply 'I dont know'.

Context: {context}
Question: {question}
"""
prompt=PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

 
Answer the question based on the context below. 
If you can't answer the question, reply 'I dont know'.

Context: Here is some context
Question: Here is a question



In [6]:
chain=prompt | model | parser

In [7]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [8]:
chain.invoke(
    {
        "context": "The name I was given was Atharva",
        "question": "What is my name?"
    }
)

' Sure! Based on the context provided, your name is Atharva.'

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore=DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [17]:
retriever=vectorstore.as_retriever()
retriever.invoke("interest rate") 

[Document(page_content='Dear Sir/Madam,A2312180286\nTo,\nAtharva Sadashiv Tapkir & Sadashiv Pandharinath Tapkir & Smita Sadashiv Tapkir\nAddress : Sr No 52 1 Building No B 7 Flat No 05 Shantiban Society Near Eklavya Collage Tc Kothrud Po Serviceman Colony,\nPune, 411038, Maharashtra\nSubject: Education Loan Sanction Letter\nBased upon and relying on the facts and information provided by you in your Application Form No. A2312180286 dated 18/12/2023 seeking grant of an\nEducation Loan ("loan"), we have pleasure in conveying you our sanction of an amount of Rs. 50,00,000/- (Rupees Fifty Lakh only) (equivalent to USD\n60,233.71 approx. at exchange rate of 1 USD : 83.01 INR. ) on following terms:\nThe Educational Loan availability is subject to satisfactory completion of loan documentation. This letter of offer shall stand unilaterally revoked and\ncancelled and shall be absolutely null and void even after acceptance by you, if:\nHDFC Credila Financial Services Limited\nAuthorized signatory

In [18]:
from operator import itemgetter
chain=(
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
    | prompt 
    | model 
    | parser
)
chain.invoke({"question": "interest rate"})

'The interest rate mentioned in the sanction letter is 10.25% p.a. floating rate.'

In [19]:
itemgetter("question")({"question": "What is the interest rate?"})

'What is the interest rate?'

In [22]:
questions=[
    "What is the interest rate provided?",
    "How much loan amount is sanctioned?",
    "What is the validity of this sanction letter?",
    "what collateral is provided for the loan?",
    "What is the term of loan repayment?"
]

for question in questions:
    print(f"Question: {question}")
    for s in chain.stream({'question': question}):
        print(s, end="", flush=True)
    # print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the interest rate provided?
The interest rate provided in the sanction letter is 10.25% p.a. floating rate.
Question: How much loan amount is sanctioned?
The loan amount sanctioned is Rs. 50,00,000/- (Rupees Fifty Lakh only).
Question: What is the validity of this sanction letter?
 Based on the context provided, the validity of the sanction letter is mentioned as "This Sanction letter is valid till 10-Jan-2025".
Question: what collateral is provided for the loan?
Based on the document provided, the collateral provided for the loan is a mortgage of immovable property being Building No B 7, 1st Floor Flat/ Shop No 5, Shantiban Society Near Eklavya Collage Tc Kothrud Po Serviceman Colony, Pune, Maharashtra owned by Sadashiv Tapkir.
Question: What is the term of loan repayment?
Based on the provided context, the term of loan repayment for the education loan sanctioned to Sadashiv Pandharinath Tapkir and Smita Sadashiv Tapkir is 168 months (or 13 years and 4 months).


In [ ]:
# chain.batch([{"question": q} for q in questions])